In [ ]:
import pandas as pd
import json
import pymysql
# Relevant post: http://wesmckinney.com/blog/high-performance-database-joins-with-pandas-dataframe-more-benchmarks/

In [ ]:
conf = json.load(open('./config.json'))

In [ ]:
period = 26
year = 17

In [ ]:
cnx = pymysql.connect(host=conf['host'], user=conf['user'], password=conf['password'], database=conf['database'])

In [ ]:
# solo estudiantes y voluntarios
people = pd.read_sql('select id, type, concat(firstname, \' \', lastname) as fullname, email  from sophia_people where type in (1,2)', 
                     cnx, 
                     index_col='id')
people.fullname =  people.fullname.str.title().str.strip() #.str.replace('[Cc]opia$','')

In [ ]:
people.head()

In [ ]:
# importa para el calculo, toca saber cada estudiante en qué cursos está inscrito
inscription = pd.read_sql('select `group`, student from sophia_academic_course_group_inscriptions where period={}'.format(period), cnx)

In [ ]:
inscription.head()

In [ ]:
course_groups = pd.read_sql('select id, name, course, teacher from sophia_academic_course_groups', cnx, index_col='id')

In [ ]:
# importa para asignar la responsabilidad a un profesor, no para el calculo
course_groups.tail()

In [ ]:
# solo importa por el nombre
courses = pd.read_sql('select id, name, subject from sophia_academic_courses where year={}'.format(year), cnx, index_col='id')

In [ ]:
courses.head()

In [ ]:
# solo importa para el nombre
subjects = pd.read_sql('select id, area, name, category from sophia_academic_subjects', cnx, index_col='id')

In [ ]:
subjects.head()

In [ ]:
subjects.category.unique()

In [ ]:
# tiene el estudiante y el curso, solo tiene las notas digitadas, no las no digitadas. ie, el value no hay nans
g_indicators = pd.read_sql('select * from sophia_academic_grading_indicators where period={}'.format(period),cnx)

In [ ]:
g_indicators.head()

In [ ]:
g_indicators[g_indicators.value.isnull()]

In [ ]:
c_indicators = pd.read_sql('select id, name, course, weight from sophia_academic_course_indicators where period={}'.format(period),cnx, index_col='id')
c_indicators.head()

In [ ]:
course_indicator_evaluators = pd.read_sql('select id, indicator, name, weight, `group` from sophia_academic_course_indicator_evaluators',cnx, index_col='id')

In [ ]:
course_indicator_evaluators.head()

In [ ]:
evaluations = pd.read_sql('select id, evaluator, name from sophia_academic_evaluations', cnx, index_col='id')
evaluations.head()

In [ ]:
evaluations_grades = pd.read_sql('select student, evaluation, value from sophia_academic_grading_evaluations', cnx)
evaluations_grades.head()

-----------------------

In [ ]:
milena = people.loc[1630]

In [ ]:
grupo_m = inscription[inscription.student == milena.name] 

In [ ]:
grupo_m # tres materias

In [ ]:
lecto_octavo_a = course_groups.loc[3822] #8A (evaluaciones) más especifico de todos 
lecto_octavo_a

In [ ]:
people.loc[lecto_octavo_a.teacher] # profesor

In [ ]:
lecto_octavo = courses.loc[lecto_octavo_a.course] #8 Lectoescrituras de 8 (indicadores)
lecto_octavo

In [ ]:
lecto = subjects.loc[lecto_octavo.subject] # Lectoescritura, No LE de 8A
lecto

In [ ]:
indicators_lecto_octavo = c_indicators[c_indicators.course==lecto_octavo.name]
print(indicators_lecto_octavo)
indicators_l_o_tareas = indicators_lecto_octavo.iloc[2]
indicators_l_o_tareas

In [ ]:
i_l_o_t_actividades = course_indicator_evaluators[(course_indicator_evaluators.indicator==indicators_l_o_tareas.name) &
                            (course_indicator_evaluators.group==lecto_octavo_a.name)] # a nivel de 8vo
print(i_l_o_t_actividades)
ilot_tarea1 = i_l_o_t_actividades.iloc[0]
ilot_tarea1

In [ ]:
partes_actividad = evaluations

In [ ]:
evaluations[evaluations.evaluator==ilot_tarea1.name]

In [ ]:
# estudiantes pueden estar activos(matriculado, pago, papeles al dia, todo bien) - se califican
#                          inscritos (también se califican) deben papeles etc - se califican
#                          retirados, es decir dejaron de venir (avisado o no) - no se califican, toca ignorar sus notas

In [ ]:
tables = pd.read_sql('show tables', cnx)

In [ ]:
for table_name in list(filter(lambda x: 'enr' in x,tables.Tables_in_phidias_colombiacrece)):
    print(pd.read_sql_query('select count(*) from {}'.format(table_name),cnx), table_name)

In [ ]:
enrollments = pd.read_sql('select * from sophia_course_section_enrollments', cnx, parse_dates=['date'])
enrollments

In [ ]:
enrollments[enrollments.person==milena.name]

In [ ]:
enrollments.status.value_counts()

In [ ]:
enrollments[enrollments.status==32]

In [ ]:
enrollments[enrollments.person==1072]

In [ ]:
# nuevp = 2
# retirado = 4
# inscrito = 32



In [ ]:
for table_name in list(filter(lambda x: 'sect' in x,tables.Tables_in_phidias_colombiacrece)):
    print(pd.read_sql_query('select count(*) from {}'.format(table_name),cnx), table_name)

In [ ]:
sections = pd.read_sql('select * from sophia_course_sections',cnx, index_col='id')

In [ ]:
sections[sections.year == year]